# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [466]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from prettytable import PrettyTable

#### Creating list of filepaths to process original event csv data files

In [467]:
print(os.getcwd())
filepath = os.getcwd() + '/event_data'
for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [468]:
full_data_rows_list = [] 

for f in file_path_list:
    
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile:  
        csvreader = csv.reader(csvfile) 
        next(csvreader)     
        for line in csvreader:
            full_data_rows_list.append(line)
            
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [469]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [470]:
from cassandra.cluster import Cluster

cluster = Cluster()

session = cluster.connect()

#### Create Keyspace

In [471]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [472]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [473]:
def insert_data(query, session, values, dtypes, file):
    """
    Pipeline to insert data to a Apache Casssandra table from CSV file    
    Args:
        query (str): INSERT statement query
        session (object): Apache Casssandra session
        values (list): list of indicies to pass to INSERT statement
        dtypes (list): list of types to convert `values`
        file (str): CSV file path   
    Returns:
        None
    """
    assert(len(values) ==  len(dtypes))
    with open(file, encoding = 'utf8') as f:
        csvreader = csv.reader(f)
        next(csvreader) # skip header
        for line in csvreader:
            session.execute(query, tuple([dtype(line[x]) for x, dtype in zip(values, dtypes)]))

### Table created for Query 1

### For this query we have considered composite partition key as sessionId and itemInSession to filter data based on sessionId and itemInSession

In [474]:
query = "CREATE TABLE IF NOT EXISTS music_app_history "
query = query + "(sessionId int,itemInSession int,artist text, song text, length double,,\
                 PRIMARY KEY ((sessionId,itemInSession)))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [475]:
file = 'event_datafile_new.csv'
# query
query = "INSERT INTO music_app_history (sessionId, itemInSession, artist, song, length)"
query = query + " VALUES (%s, %s, %s, %s, %s)"
# values and data types
values = [8, 3, 0, 9, 5]
dtypes = [int, int, str, str, float]
# insert to table
insert_data(query, session, values, dtypes, file)

### 1. Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

In [476]:
query = "select artist,song,length from music_app_history where sessionId = 338 and itemInSession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

t = PrettyTable(['Artist', 'Song', 'Length'])
for row in rows:
    t.add_row([row.artist, row.song, row.length])
print(t)

+-----------+---------------------------------+----------+
|   Artist  |               Song              |  Length  |
+-----------+---------------------------------+----------+
| Faithless | Music Matters (Mark Knight Dub) | 495.3073 |
+-----------+---------------------------------+----------+


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

### Table created for Query2.
### For this query we need user details based on userId and sessionId hence userId and sessionId are considered as COMPOSITE KEY  and itemInSession as ClUSTERING key since the results need to be sorted by clustering colum itemInSession

In [477]:
query1 = "CREATE TABLE IF NOT EXISTS user_details"
query1 = query1 + "(userId int,sessionId int,itemInSession int,artist text, song text,firstName text,lastName text,\
                    PRIMARY KEY  ((userId, sessionId), itemInSession)) \
                    WITH CLUSTERING ORDER BY (itemInSession ASC);"
try:
    session.execute(query1)
except Exception as e:
    print(e)              

In [478]:
file = 'event_datafile_new.csv'
# query
query = "INSERT INTO user_details (userId,sessionId,itemInSession,artist,song,\
                     firstName,lastName)"
query = query + " VALUES (%s, %s, %s, %s, %s,%s,%s)"
values = [10, 8, 3, 0, 9, 1, 4]
dtypes = [int, int, int, str, str, str,str]
# insert to table
insert_data(query, session, values, dtypes, file)

### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [479]:
query1 = "select artist,song,firstname,lastname from user_details where userId=10 and sessionId=182"
try:
    rows = session.execute(query1)
except Exception as e:
    print(e)   
t = PrettyTable(['Artist', 'Song', 'FirstName','LastName'])
for row in rows:
    t.add_row([row.artist, row.song, row.firstname,row.lastname])
print(t)

+-------------------+------------------------------------------------------+-----------+----------+
|       Artist      |                         Song                         | FirstName | LastName |
+-------------------+------------------------------------------------------+-----------+----------+
|  Down To The Bone |                  Keep On Keepin' On                  |   Sylvie  |   Cruz   |
|    Three Drives   |                     Greece 2000                      |   Sylvie  |   Cruz   |
| Sebastien Tellier |                      Kilometer                       |   Sylvie  |   Cruz   |
|   Lonnie Gordon   | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) |   Sylvie  |   Cruz   |
+-------------------+------------------------------------------------------+-----------+----------+


### Table created for Query3

### For this query we need users who listened to a patricular song therefore to uniquely identify the users i have used userId and song as COMPOSITE KEY

In [480]:
query3 = "CREATE TABLE IF NOT EXISTS user_info"
query3 = query3 + "(song text,userId int,firstName text, lastName text, \
                    PRIMARY KEY (song, userId))"
try:
    session.execute(query3)
except Exception as e:
    print(e)       

In [481]:
file = 'event_datafile_new.csv'
# query
query = "INSERT INTO user_info (song, userId,firstName,lastName)"
query = query + " VALUES (%s, %s, %s, %s)"
values = [9,10,1,4]
dtypes = [str,int,str,str]
# insert to table
insert_data(query, session, values, dtypes, file)

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [482]:
query3 = "select firstname,lastname from user_info where song='All Hands Against His Own'"
try:
    rows = session.execute(query3)
except Exception as e:
    print(e)
t = PrettyTable(['FirstName','LastName'])
for row in rows:
    t.add_row([row.firstname,row.lastname])
print(t)

+------------+----------+
| FirstName  | LastName |
+------------+----------+
| Jacqueline |  Lynch   |
|   Tegan    |  Levine  |
|    Sara    | Johnson  |
+------------+----------+


### Drop the tables before closing out the sessions

In [483]:
query = "drop table music_app_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [484]:
query = "drop table user_details"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [485]:
query = "drop table user_info"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [486]:
session.shutdown()
cluster.shutdown()